In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.utils.test_scripts import *
from pkg.planning.pddlstream.plan_rnb import *

In [3]:
rtype = "panda"
dat_root = "stowing-deep"
res_root = "stowing-deep-result"
dat_dir = "20210907-173519"
file_option = "obj_1c_obs0"
data_idx = 1
cname = "Full"

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100
MAX_SKELETONS = 10

GRASP_SAMPLE = 100
STABLE_SAMPLE = 100
SEARCH_SAMPLE_RATIO = 10

VISUALIZE = True
SHOW_STATE = True
PLAY_RESULT = True
USE_PYBULLET_GUI = True



CLEARANCE = 1e-3
TOOL_NAME="grip0"
ROBOT_TYPE = {e.name: e for e in RobotType}[rtype]

In [4]:
DATASET_PATH = create_data_dirs(dat_root, rtype, dat_dir)
RESULTSET_PATH = create_data_dirs(res_root, rtype, dat_dir)
print("-"*50)
print("DATASET_PATH: {}".format(DATASET_PATH))
print("RESULTSET_PATH: {}".format(RESULTSET_PATH))
print("-"*50)

--------------------------------------------------
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep/panda/20210907-173519
RESULTSET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep-result/panda/20210907-173519
--------------------------------------------------


In [5]:
ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene
crob.home_pose = HOME_POSE
crob.home_dict = list2dict(HOME_POSE, gscene.joint_names)

fname = "data_%s_%02d.pkl" % (file_option, data_idx)
print(fname)
file_gtems = os.path.join(DATASET_PATH, fname)
initial_state = load_saved_scene(pscene, file_gtems, VISUALIZE=VISUALIZE)

gscene.NAME_DICT['obj_0'].color = (1,0,0,1)
gscene.update_markers_all()

mplan = MoveitPlanner(pscene)
checkers = get_checkers_by_case_name(cname, pscene)

mplan.motion_filters = checkers

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
data_obj_1c_obs0_01.pkl


## Solve in PDDLStream

In [ ]:
obj_pscene = pscene.subject_dict[pscene.subject_name_list[0]]
obj_pscene.geometry.color = (0.8, 0.2, 0.2, 1)
gscene.update_markers_all()
goal_pairs=[(obj_pscene.oname, 'gp')]

gtimer = GlobalTimer.instance()
gtimer.reset()

res, plan, log_dict = solve_in_pddlstream(pscene, mplan, ROBOT_NAME, TOOL_NAME, HOME_POSE, goal_pairs, 
                        TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS, 
                        GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO, 
                        use_pybullet_gui=USE_PYBULLET_GUI)

# save_pickle(os.path.join(RESULTSET_PATH, "result_%s_%02d_%s.pkl" % (file_option, data_idx, cname)), log_dict)

print("------- Result {} ({}): {} s -------".format(fname, cname, log_dict["plan_time"]))
print("==========================================================")
print("==========================================================")
print(gtimer)
print("==========================================================")
print("==========================================================")


In [ ]:
if VISUALIZE and PLAY_RESULT and res:
    play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
                   body_names=log_dict['body_names'], plan=plan, SHOW_PERIOD=0.01)


## Old version

In [6]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
generate table - Geometry
generate table - Handle
 generate table - Object
generate table - Binder


In [8]:
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

   Use a production WSGI server instead.
 * Debug mode: off


In [9]:
from_state = initial_state.copy(pscene)
from_state.Q = HOME_POSE
goal_nodes = [('gp',)]
ppline.search(from_state, goal_nodes, verbose=True, display=False, 
              timeout_loop=100, multiprocess=False, timeout=5, add_homing=False)

try: 0 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecke

result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: ReachCheck

result: 2 - ('wp',)->('grip0',) = fail
try: 2 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 2 - ('wp',)->('grip0',) = fail
try: 2 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 2 - ('wp',)->('grip0',) = fail
try: 1 - ('grip0',)->('wp',)
try transition motion
transition motion tried: True
result: 1 - ('grip0',)->('wp',) = success
branching: 1->3 (44.63/100.0 s, steps/err: 76(89.0810489655 ms)/0.0015833710617)
try: 0 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('wp',)->('grip0',) = fail
try: 3 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 3 - ('wp',)->('grip0',) = fail
try: 1 - ('grip0',)->('wp',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0',)->('wp',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('gri

transition motion tried: False
Motion Plan Failure
result: 4 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 2 - ('wp',)->('grip0',)
Motion Filer Failure: ReachChecker
result: 2 - ('wp',)->('grip0',) = fail
try: 1 - ('grip0',)->('wp',)
try transition motion
transition motion tried: True
result: 1 - ('grip0',)->('wp',) = success
branching: 1->10 (70.04/100.0 s, steps/err: 39(85.9181880951 ms)/0.00128275757773)
try: 5 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 5 - ('wp',)->('grip0',) = fail
try: 8 - ('wp',)->('grip0',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 8 - ('wp',)->('grip0',) = fail
try: 5 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 5 - ('wp',)->('grip0',) = fail
try: 1 - ('grip0',)->('wp',)
try transition motion
transition motion tried: True
result: 1 - ('grip0',)->('wp',) 

In [10]:
snode_schedule = ppline.tplan.get_best_schedule()

('grip0',)->('wp',)


IndexError: list index out of range

button action done


In [ ]:
ppline.play_schedule(snode_schedule)